<a href="https://www.kaggle.com/code/mikedelong/aggregate-sentiment-analysis?scriptVersionId=160556877" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
from transformers import pipeline

In [2]:
import pandas as pd
from os.path import basename
from glob import glob
from arrow import now

FILES = glob(pathname='/kaggle/input/genaitweets10k/*.csv')

time_start = now()
objs = []
for input_file in FILES:
    print('reading {}'.format(input_file))
    this_df = pd.read_csv(filepath_or_buffer=input_file, names=['text'], sep='\t')
    this_df['file'] = [basename(input_file).replace('.csv', '')] * len(this_df)
    objs.append(this_df)

df = pd.concat(objs=objs)
df['no hash tags'] = df['text'].apply(func = lambda x: ' '.join([token for token in x.split() if not token.startswith('#')]))
print('loaded {} rows from {} files in {}'.format(len(df), len(objs), now() - time_start))

reading /kaggle/input/genaitweets10k/Election2020AI.csv
reading /kaggle/input/genaitweets10k/SelfDrivingAI.csv
reading /kaggle/input/genaitweets10k/GoTJulyAI.csv
reading /kaggle/input/genaitweets10k/financeAI.csv
reading /kaggle/input/genaitweets10k/IndianTrafficIncidentsAI.csv
reading /kaggle/input/genaitweets10k/Covid19AI.csv
reading /kaggle/input/genaitweets10k/FIFA2022AI.csv
reading /kaggle/input/genaitweets10k/MeTooAI.csv
reading /kaggle/input/genaitweets10k/AvengersAI.csv
reading /kaggle/input/genaitweets10k/ClimateChangeAI.csv
reading /kaggle/input/genaitweets10k/weatherAI.csv
reading /kaggle/input/genaitweets10k/AirlineCritique.csv
reading /kaggle/input/genaitweets10k/GymnasticsOlympicsAI.csv
reading /kaggle/input/genaitweets10k/BitcoinAI.csv
reading /kaggle/input/genaitweets10k/FathersDayAI.csv
reading /kaggle/input/genaitweets10k/gpt4AI.csv
reading /kaggle/input/genaitweets10k/SamsugnMobileAccessoriesAI.csv
reading /kaggle/input/genaitweets10k/palestine2020-2021AI.csv
reading

In [3]:
MODELS = [
    'bhadresh-savani/distilbert-base-uncased-emotion',
    'cardiffnlp/twitter-roberta-base-sentiment',
    'nlptown/bert-base-multilingual-uncased-sentiment',
]

# this takes 15-20 minutes without an accelerator so sit tight
time_start = now()
pipe = pipeline(task='sentiment-analysis', model=MODELS[2],)

df['result'] = df['text'].apply(pipe)
df['label'] = df['result'].apply(func=lambda x: x[0]['label'])
df['score'] = df['result'].apply(func=lambda x: x[0]['score'])
df = df.drop(columns=['result'])
print('sentiment analysis done for {} rows in {}'.format(len(df), now() - time_start))

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

sentiment analysis done for 10505 rows in 0:16:06.656080


In [4]:
from plotly.express import histogram
histogram(data_frame=df, x='file',  color='label')

In [5]:
histogram(data_frame=df, x='file',  color='label', y='score')

Maybe a strip/swarm plot will let us visualize all the data in a browsable way.

In [6]:
from plotly.express import strip
strip(data_frame=df, x='file', y='score', color='label', hover_name='no hash tags', height=1800, range_y=[0.2, 1])